In [1]:
# 개와 고양이 데이터세트를 다운로드합니다.
!wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
# 현재 경로에 압축을 해제합니다.
!unzip cats_and_dogs_filtered.zip

--2021-08-06 17:01:04--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.128, 142.250.141.128, 2607:f8b0:4023:c0d::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip’

cats_and_dogs_filte 100%[===================>]  65.43M  94.7MB/s    in 0.7s    

2021-08-06 17:01:05 (94.7 MB/s) - ‘cats_and_dogs_filtered.zip’ saved [68606236/68606236]

Archive:  cats_and_dogs_filtered.zip
   creating: cats_and_dogs_filtered/
  inflating: cats_and_dogs_filtered/vectorize.py  
   creating: cats_and_dogs_filtered/validation/
   creating: cats_and_dogs_filtered/train/
   creating: cats_and_dogs_filtered/validation/dogs/
  inflating: cats_and_dogs_filtered/validation/dogs/dog.2127.jpg  
  inflating: cats_and_dogs_filtered/validation/dog

In [2]:
# 실습에 필요한 라이브러리를 불러옵니다.
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
# 학습 세트에 적용할 데이터 변환을 설정합니다.
train_config = transforms.Compose([transforms.Resize((224, 224)),
                                   transforms.RandomHorizontalFlip(),            
                                   transforms.ToTensor()
])

# 테스트 세트에 적용할 데이터 변환을 설정합니다.
test_config = transforms.Compose([transforms.Resize((224, 224)),
                                  transforms.ToTensor()
])

# 이미지를 불러와서 위의 설정을 반영한 데이터세트 자료구조를 만듭니다.
train_dset = datasets.ImageFolder('./cats_and_dogs_filtered/train/', train_config)
test_dset = datasets.ImageFolder('./cats_and_dogs_filtered/validation/', test_config)

# 한 번에 32개의 데이터 샘플을 배치로 사용하는 데이터로더를 생성합니다.
train_loader = DataLoader(train_dset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dset, batch_size=32, shuffle=False)

In [4]:
# 사전 학습 모델인 VGG16 모델 객체를 생성하고 가중치를 불러옵니다.
model = models.vgg16(pretrained=True)

# 모델의 구조를 확인합니다.
print(model)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth



VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=

In [5]:
# 모델의 가중치를 더이상 학습하지 않도록 설정합니다.
for param in model.features.parameters():
    param.require_grad = False

In [6]:
# 출력층을 한 개의 노드를 가진 전결합층으로 교체합니다.
model.classifier[-1] = nn.Sequential(
    nn.Linear(model.classifier[-1].in_features, 1),
    nn.Sigmoid()
)

# 그래픽카드 사용이 가능할 경우 그래픽카드로 연산하도록 설정합니다.
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 모델의 연산을 그래픽 카드에서 하도록 설정합니다.
print(model.to(device))

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [7]:
# 이진 크로스 엔트로피(Binary Cross Entropy) 손실 함수 객체를 생성합니다.
criterion = nn.BCELoss()

# 확률적 경사 하강법 옵티마이저 객체를 생성합니다.
optimizer = optim.Adam(model.parameters(), lr=1e-6)

In [8]:
# 학습 함수를 정의합니다.
def train(model, criterion, optimizer, loader):
  # 현재 에포크의 오차와 정확도를 저장할 변수를 생성합니다.
  epoch_loss = 0
  epoch_acc = 0

  # 모델을 학습 모드로 설정합니다.
  model.train()

  # 배치 학습을 실행합니다.
  for X_batch, y_batch in loader:
    # 입력 데이터와 타깃을 준비합니다.
    X_batch, y_batch = X_batch.to(device), y_batch.to(device).float().view(-1, 1)
    # 기울기를 초기화합니다.
    optimizer.zero_grad()
    # 모델을 사용해 타깃을 추론합니다.
    hypothesis = model(X_batch)
    # 손실 함수로 오차를 계산합니다.
    loss = criterion(hypothesis, y_batch)        
    # 기울기를 계산합니다.
    loss.backward()
    # 경사 하강법으로 가중치를 수정합니다.
    optimizer.step()    
    # 확률로 표현된 추론값을 타깃을 나타내는 정수 형태로 변환합니다.
    y_predicted = hypothesis >= 0.5
    # 현재 배치의 정확도를 계산합니다.
    acc = (y_predicted == y_batch).float().mean()
    # 현재 배치의 오차와 정확도를 저장합니다.
    epoch_loss += loss.item()
    epoch_acc += acc.item()

  # 현재 에포크의 오차와 정확도를 반환합니다.
  return epoch_loss / len(loader), epoch_acc / len(loader)

In [9]:
# 평가 함수를 정의합니다.
def evaluate(model, criterion, loader):
  # 현재 에포크의 오차와 정확도를 저장할 변수를 생성합니다.
  epoch_loss = 0
  epoch_acc = 0

  # 모델을 평가 모드로 설정합니다.
  model.eval()

  with torch.no_grad():
    # 배치 단위로 추론을학습을 실행합니다.
    for X_batch, y_batch in loader:
      # 입력 데이터와 타깃을 그래픽 카드로 연산하도록 준비합니다.
      X_batch, y_batch = X_batch.to(device), y_batch.to(device).float().view(-1, 1)
      # 모델을 사용해 타깃을 추론합니다.
      hypothesis = model(X_batch)
      # 손실 함수로 오차를 계산합니다.
      loss = criterion(hypothesis, y_batch)
      # 확률로 표현된 추론값을 타깃을 나타내는 정수 형태로 변환합니다.
      y_predicted = hypothesis >= 0.5
      # 현재 배치의 정확도를 계산합니다.
      acc = (y_predicted == y_batch).float().mean()
      # 현재 배치의 오차와 정확도를 저장합니다.
      epoch_loss += loss.item()
      epoch_acc += acc.item()

    # 현재 에포크의 오차와 정확도를 반환합니다.
    return epoch_loss / len(loader), epoch_acc / len(loader)

In [10]:
# 10회에 걸쳐 모델을 학습시킵니다.
n_epochs = 10
for epoch in range(1, n_epochs+1):
  # 모델을 학습시킵니다.
  loss, acc = train(model, criterion, optimizer, train_loader)

  # 모델을 평가합니다.
  test_loss, test_acc = evaluate(model, criterion, test_loader)

  # 현재 에포크의 학습 결과를 출력합니다.
  print('epoch: {}, loss: {:.3f}, acc: {:.2f}, test_loss: {:.3f}, test_acc: {:.3f}'.format(
      epoch, loss, acc, test_loss, test_acc
  ))


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch: 1, loss: 0.695, acc: 0.53, test_loss: 0.598, test_acc: 0.801
epoch: 2, loss: 0.546, acc: 0.78, test_loss: 0.456, test_acc: 0.910
epoch: 3, loss: 0.396, acc: 0.88, test_loss: 0.296, test_acc: 0.938
epoch: 4, loss: 0.254, acc: 0.93, test_loss: 0.181, test_acc: 0.956
epoch: 5, loss: 0.166, acc: 0.94, test_loss: 0.125, test_acc: 0.959
epoch: 6, loss: 0.121, acc: 0.96, test_loss: 0.095, test_acc: 0.971
epoch: 7, loss: 0.100, acc: 0.96, test_loss: 0.082, test_acc: 0.971
epoch: 8, loss: 0.082, acc: 0.97, test_loss: 0.072, test_acc: 0.973
epoch: 9, loss: 0.065, acc: 0.98, test_loss: 0.065, test_acc: 0.974
epoch: 10, loss: 0.058, acc: 0.98, test_loss: 0.061, test_acc: 0.976
